#### Name : Mani Botla
#### Date : 8/12/2025
#### Cleaning the dataset for analysis.

#### Import pandas and load the csv file

In [1]:
import pandas as pd
df = pd.read_csv("trainSet.csv")

#### Checking outliers and summary statistics
`.describe()` : summary stats for all columns - to check if there are extreme values in numeric or date columns

In [2]:

df.describe(include="all")


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
count,1.048575e+06,1048575,1048575,1.048575e+06,1048575,1.048575e+06,1048575,1048575,1048575
unique,NaN,425,425,NaN,3,NaN,5,195,195538
top,NaN,8/8/2016,8/10/2016,NaN,desktop,NaN,Gondal,Cobra Island,3635431_3
freq,NaN,8621,8514,NaN,677799,NaN,482468,123839,48
mean,3.213420e+06,NaN,NaN,3.347743e+04,NaN,5.720926e+03,NaN,NaN,NaN
std,1.690987e+06,NaN,NaN,1.957905e+04,NaN,3.963644e+03,NaN,NaN,NaN
min,2.900000e+01,NaN,NaN,2.000000e+00,NaN,5.000000e+00,NaN,NaN,NaN
25%,1.846553e+06,NaN,NaN,1.701300e+04,NaN,1.013000e+03,NaN,NaN,NaN
50%,3.250225e+06,NaN,NaN,3.233900e+04,NaN,6.309000e+03,NaN,NaN,NaN
75%,4.666156e+06,NaN,NaN,5.112800e+04,NaN,9.924000e+03,NaN,NaN,NaN


#### Check for missing values : checks how many NaN values are in each column

In [3]:

missing_summary = df.isna().sum()
missing_summary


user_id           0
checkin           0
checkout          0
city_id           0
device_class      0
affiliate_id      0
booker_country    0
hotel_country     0
utrip_id          0
dtype: int64

#### Results : 0 NaN values were found.

#### Standardizing column names and making sure all column names are replaced with underscore if they have a space in the name

In [4]:
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
df.head()


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,1000027,8/13/2016,8/14/2016,8183,desktop,7168,Elbonia,Gondal,1000027_1
1,1000027,8/14/2016,8/16/2016,15626,desktop,7168,Elbonia,Gondal,1000027_1
2,1000027,8/16/2016,8/18/2016,60902,desktop,7168,Elbonia,Gondal,1000027_1
3,1000027,8/18/2016,8/21/2016,30628,desktop,253,Elbonia,Gondal,1000027_1
4,1000033,4/9/2016,4/11/2016,38677,mobile,359,Gondal,Cobra Island,1000033_1


#### Check Date Columns for Validity : ensure that `checkin` and `checkout` are valid dates and see if any rows have invalid or missing dates. Count invalid dates.


In [5]:
date_cols = ["checkin", "checkout"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

df[date_cols].isna().sum()


checkin     0
checkout    0
dtype: int64

#### Result : 0 invalid dates was the output. 

### Identify Logical Date Errors -  check if any `checkout` dates are earlier than `checkin` dates.


In [6]:
invalid_date_ranges = df[df["checkout"] < df["checkin"]]
invalid_date_ranges


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id


#### Result : The code did not return any row so there no logical date errors.

#### Find any outliers using IQR method: 

In [7]:
import numpy as np

for col in df.select_dtypes(include=np.number).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers")


user_id: 0 outliers
city_id: 0 outliers
affiliate_id: 0 outliers


#### 0 outliers were found for the  numeric columns.

#### `.head()` : display first few rows of cleaned dataset

In [8]:
df.head()

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,1000027,2016-08-13,2016-08-14,8183,desktop,7168,Elbonia,Gondal,1000027_1
1,1000027,2016-08-14,2016-08-16,15626,desktop,7168,Elbonia,Gondal,1000027_1
2,1000027,2016-08-16,2016-08-18,60902,desktop,7168,Elbonia,Gondal,1000027_1
3,1000027,2016-08-18,2016-08-21,30628,desktop,253,Elbonia,Gondal,1000027_1
4,1000033,2016-04-09,2016-04-11,38677,mobile,359,Gondal,Cobra Island,1000033_1


#### Save to new csv file.

In [9]:

df.to_csv("train_set_cleaned.csv", index=False)
